## Load all session data

In [1]:
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))
import pandas as pd
from ephys import *
from session_utils import find_all_sessions

# Load pickled data
df_all_sessions = pd.read_pickle('/home/isabella/Documents/isabella/jake/ephys_analysis/processed_data/df_all_cells.pkl')

# Load all session names and paths as dict
session_dict = find_all_sessions(sheet_path = 'https://docs.google.com/spreadsheets/d/1_Xs5i-rHNTywV-WuQ8-TZliSjTxQCCqGWOD2AL_LIq0/edit#gid=0',
                                 data_path = '/home/isabella/Documents/isabella/jake/recording_data',
                                 sorting_suffix = 'sorting_ks2_custom')

# Initialise DataFrame for ephys objects
df_all_sessions = pd.DataFrame(data = None, index = session_dict.keys(), columns = ['ephys_object'], dtype='object')

for i, session_path in enumerate(session_dict.values()):
    session = list(session_dict.keys())[i]

    # Create ephys object for session and add to dataframe
    obj = ephys(recording_type = 'nexus', path = session_path)
    df_all_sessions.at[list(session_dict.keys())[i], 'ephys_object'] = obj

# # Drop rows with no included clusters
# df_all_cells = df_all_cells.dropna()
# print(f'{len(df_all_cells.index)} sessions retained')

# TEMP DROP ALL BUT ONE SESSION
# df_all_cells = df_all_cells.iloc[10:11,:]

df_all_sessions.head()

,ephys_object
230503_r1354,<ephys.ephys object at 0x7f88eb96ccd0>
230504_r1354,<ephys.ephys object at 0x7f88eb43c410>
230505_r1354,<ephys.ephys object at 0x7f88eb40b090>
230506_r1354,<ephys.ephys object at 0x7f88eb43c150>
230507_r1354,<ephys.ephys object at 0x7f88eb40b750>


## Select times when animal is in choice or return arms

In [2]:
from analysis.position_analysis import assign_sectors

# Loop through sessions
trials_loaded = 0
for row, obj in enumerate(df_all_sessions.loc[:, 'ephys_object']):
        
    # Load position for t-maze trials
    for trial_iterator, trial in enumerate(obj.trial_list):
        if 't-maze' in trial:
            obj.load_pos(trial_iterator, reload_flag = False)
            trials_loaded +=1
            
            # Assign sectors to positions samples
            sector_numbers = np.array(assign_sectors(obj.pos_data[trial_iterator]['xy_position'].T, pos_header = obj.pos_data[trial_iterator]['header']))
            
            # Define central and return arm position sectors
            central_sectors = [6, 7]
            return_sectors = [2, 3, 10, 11]
            
            # Find position samples where the animal is in each arm type
            central_samples = np.where(np.isin(sector_numbers, central_sectors))[0]
            return_samples = np.where(np.isin(sector_numbers, return_sectors))[0]
            
            # Get position sampling rate
            pos_sample_rate = obj.pos_data[trial_iterator]['pos_sampling_rate']
            
            #Convert these samples into times (in s) through the trial (divide by sample rate), then add to obj
            obj.pos_data[trial_iterator]['central_times'] = central_samples / pos_sample_rate
            obj.pos_data[trial_iterator]['return_times'] = return_samples / pos_sample_rate            
            
            df_all_sessions.at[list(session_dict.keys())[row], 'ephys_object'] = obj
                      
print(f'{trials_loaded} t-maze trials loaded')

Loading pos file: /home/isabella/Documents/isabella/jake/recording_data/r1354/2023-05-03/230503_r1354_raw_t-maze_1.pos
Real PPM artifically set to 615 (t-maze default)


/data/isabella/jake/ephys_analysis/pyScan/analysis/../postprocessing/postprocess_pos_data.py:219: RuntimeWarning: invalid value encountered in remainder
  dir_disp = np.mod(np.arctan2(-pos.iloc[1, 1:].values + pos.iloc[1, :-1].values, pos.iloc[0, 1:].values - pos.iloc[0, :-1].values) * 180 / np.pi, 360)


Loading pos file: /home/isabella/Documents/isabella/jake/recording_data/r1354/2023-05-04/230504_r1354_raw_t-maze_1.pos
Real PPM artifically set to 615 (t-maze default)
Loading pos file: /home/isabella/Documents/isabella/jake/recording_data/r1354/2023-05-05/230505_r1354_raw_t-maze_1.pos
Real PPM artifically set to 615 (t-maze default)
Loading pos file: /home/isabella/Documents/isabella/jake/recording_data/r1354/2023-05-05/230505_r1354_raw_t-maze_2.pos
Real PPM artifically set to 615 (t-maze default)
Loading pos file: /home/isabella/Documents/isabella/jake/recording_data/r1354/2023-05-06/230506_r1354_raw_t-maze_1.pos
Real PPM artifically set to 615 (t-maze default)
Loading pos file: /home/isabella/Documents/isabella/jake/recording_data/r1354/2023-05-06/230506_r1354_raw_t-maze_2.pos
Real PPM artifically set to 615 (t-maze default)
Loading pos file: /home/isabella/Documents/isabella/jake/recording_data/r1354/2023-05-07/230507_r1354_raw_t-maze_1.pos
Real PPM artifically set to 615 (t-maze d

## Load theta phase and cycle for each t-maze trial

In [8]:
from analysis.get_theta_frequencies import get_theta_frequencies
from analysis.get_theta_phase import get_theta_phase

lfp_sampling_rate = 1000
# channels_to_load = [35, 58, 37, 56, 39, 55, 40, 57, 38, 54, 41, 53, 42] # From generate 5x12/16 probe script, corresponding to central shank @0um to -1200um
channels_to_load = [35] # From generate 5x12/16 probe script, corresponding to central shank @0um

# Loop through sessions
for row, obj in enumerate(df_all_sessions.loc[:, 'ephys_object']):
        
    # Load central and return times for t-maze trials
    for trial_iterator, trial in enumerate(obj.trial_list):
        if 't-maze' in trial:

            # Load LFP for trial from contact on central shank at 0um (channel 15 on the 5x12/16 Buz probe)
            obj.load_lfp(trial_iterator,
                         sampling_rate = lfp_sampling_rate, 
                         channels  = channels_to_load, 
                         reload_flag = False,
                         scale_to_uv = True)
            
            # Find peak theta frequencies for channel @ 0um and make dict of {channel: theta_freq}
            theta_freqs = get_theta_frequencies(obj.lfp_data[trial_iterator]['data'], lfp_sampling_rate)
            theta_freqs = dict(zip(channels_to_load, theta_freqs))

            # Initialise theta phase array
            theta_phase = [None] * len(channels_to_load)
            cycle_numbers = [None] * len(channels_to_load)

            # Loop through each included channel
            for channel_iterator, channel in enumerate(channels_to_load):

                # Get peak theta frequency
                theta_freq = theta_freqs[channel]

                # Get LFP data for channel
                lfp_data_for_channel = obj.lfp_data[trial_iterator]['data'][:, channel_iterator]

                # Calculate theta phase
                theta_phase[channel_iterator], cycle_numbers[channel_iterator] = get_theta_phase(lfp_data_for_channel, lfp_sampling_rate, theta_freq)


            # Add theta phase data to object
            obj.lfp_data[trial_iterator]['theta_phase'] = np.array(theta_phase).T
            # Add cycle numbers data to object
            obj.lfp_data[trial_iterator]['cycle_numbers'] = np.array(cycle_numbers).T
            print(f'Loaded theta phase data for trial: {trial}')
                        
    df_all_sessions.at[list(session_dict.keys())[row], 'ephys_object'] = obj


LFP data already loaded for trial 0
Loaded theta phase data for trial: 230503_r1354_raw_t-maze_1
LFP data already loaded for trial 2
Loaded theta phase data for trial: 230504_r1354_raw_t-maze_1
LFP data already loaded for trial 1
Loaded theta phase data for trial: 230505_r1354_raw_t-maze_1
LFP data already loaded for trial 3
Loaded theta phase data for trial: 230505_r1354_raw_t-maze_2
LFP data already loaded for trial 1
Loaded theta phase data for trial: 230506_r1354_raw_t-maze_1
LFP data already loaded for trial 3
Loaded theta phase data for trial: 230506_r1354_raw_t-maze_2
LFP data already loaded for trial 1
Loaded theta phase data for trial: 230507_r1354_raw_t-maze_1
LFP data already loaded for trial 1
Loaded theta phase data for trial: 230508_r1354_raw_t-maze_1
LFP data already loaded for trial 3
Loaded theta phase data for trial: 230508_r1354_raw_t-maze_2
LFP data already loaded for trial 1
Loaded theta phase data for trial: 230509_r1354_raw_t-maze_1
LFP data already loaded for tr

## For each t-maze trial - compute CSD by arm for each theta cycle, save result to dataframe

In [9]:
from analysis.get_traversal_data import get_traversal_cycles, get_data_for_traversals, drop_extreme_cycles
from analysis.bz_csd import calculate_csd_df
from scipy.interpolate import interp1d

channels_to_load = [35, 58, 37, 56, 39, 55, 40, 57, 38, 54, 41, 53, 42] # From generate 5x12/16 probe script, corresponding to central shank @0um to -1200um

# Create dataframe for output
cycle_df = pd.DataFrame(columns=['Central Cycles', 'Return Cycles']).astype(object)
    
# Loop through sessions
for obj in df_all_sessions.loc[:, 'ephys_object']:
        
    # Loop through t-maze trials
    for trial_iterator, trial in enumerate(obj.trial_list):
        
        if 't-maze' in trial:
        
            ## Get central and return arm times from object
            central_times = obj.pos_data[trial_iterator]['central_times']
            return_times = obj.pos_data[trial_iterator]['return_times']

            ## Load relevant lfp data from object
            cycle_numbers = obj.lfp_data[trial_iterator]['cycle_numbers']
            lfp_timestamps = obj.lfp_data[trial_iterator]['timestamps']
            lfp_sampling_rate = obj.lfp_data[trial_iterator]['sampling_rate']
            theta_phase = obj.lfp_data[trial_iterator]['theta_phase']
            
            ## Reload LFP data to include all relevant channels for CSD calculation. This will load 13 channels worth of LFP data 
            # This will also write over the calculated metrics stored above
            obj.load_lfp(trial_iterator,
             sampling_rate = lfp_sampling_rate, 
             channels  = channels_to_load, 
             reload_flag = True,
             scale_to_uv = True)
            
            lfp_data = obj.lfp_data[trial_iterator]['data']
            
            ## Get speed data and align to LFP timestamps
            speed_data = obj.pos_data[trial_iterator]['speed']
            pos_sampling_rate = obj.pos_data[trial_iterator]['pos_sampling_rate']
            num_data_points = len(speed_data)
            # Original timestamps at 50 Hz
            original_timestamps = np.linspace(0, num_data_points/pos_sampling_rate, num=num_data_points, endpoint=True)
            # Create interpolation function
            interpolation_function = interp1d(original_timestamps, speed_data, kind='linear')
            # Interpolate to get new data at 1000 Hz
            resampled_speed_data = interpolation_function(lfp_timestamps)
            
            ## Calculate individual arm traversals and get identity of whole theta cycles within each traversal
            central_cycles = get_traversal_cycles(central_times, cycle_numbers, lfp_timestamps, lfp_sampling_rate)
            return_cycles = get_traversal_cycles(return_times, cycle_numbers, lfp_timestamps, lfp_sampling_rate)
            
            ## Get lfp trace, theta phase, timestamps, speed, cycle index and traversal index for included individual theta cycles
            central_cycle_df = get_data_for_traversals(central_cycles, cycle_numbers, lfp_data, resampled_speed_data, channels_to_load, theta_phase, lfp_timestamps)
            return_cycle_df = get_data_for_traversals(return_cycles, cycle_numbers, lfp_data, resampled_speed_data, channels_to_load, theta_phase, lfp_timestamps)

            print(f'Added trial {trial}')
            
            ## Loop through each traversal in the dataframe and calculate CSD and gamma power, add to dataframe
            central_csd_df = calculate_csd_df(central_cycle_df)
            return_csd_df = calculate_csd_df(return_cycle_df)
            
            ## Drop the first and last theta cycle for each traversal to remove calculation artefacts
            central_csd_df = drop_extreme_cycles(central_cycle_df)
            return_cycle_df = drop_extreme_cycled(return_cycle_df)
            
            ## Filter dataframe for speed > 2.5 cm/s
            central_csd_df = central_csd_df.loc[:, central_csd_df.loc['Speed'] > 2.5]
            return_csd_df = return_csd_df.loc[:, central_csd_df.loc['Speed'] > 2.5]
            
            ## Plot CSD vs theta phase for each arm
            # Group df by phase (100 bins 0 to 2*pi)
            bins = np.linspace(0, 2 * np.pi, 101)
            binned_theta = np.digitize(df.loc[:,'Cycle Theta Phase'].values, bins)
            

            # Mean CSD for each channel in that bin
            
            # Plot using contourf as in bz_csd function


Added trial 230503_r1354_raw_t-maze_1
LFP fragment is empty. Skipping processing.
Skipping traversal 0, not enough data
LFP fragment is empty. Skipping processing.
Skipping traversal 8, not enough data
LFP fragment is empty. Skipping processing.
Skipping traversal 10, not enough data
LFP fragment is empty. Skipping processing.
Skipping traversal 12, not enough data
LFP fragment is empty. Skipping processing.
Skipping traversal 15, not enough data
LFP fragment is empty. Skipping processing.
Skipping traversal 17, not enough data
LFP fragment is empty. Skipping processing.
Skipping traversal 18, not enough data
LFP fragment is empty. Skipping processing.
Skipping traversal 23, not enough data
LFP fragment is empty. Skipping processing.
Skipping traversal 25, not enough data
LFP fragment is empty. Skipping processing.
Skipping traversal 27, not enough data
LFP fragment is empty. Skipping processing.
Skipping traversal 35, not enough data
LFP fragment is empty. Skipping processing.
Skippin

KeyError: '[43.5 59.165 69.341 69.438 70.125 71.052 71.812 71.951 72.392 72.392\n 79.438 91.74 92.5 92.5 112.886 152.174 156.801 156.985 164.407 164.407\n 164.883 171.814 173.542 174.182 174.956 175.96 178.078 185.223 187.094\n 187.608 188.873 190.302 191.454 191.794 193.198 193.589 194.882 194.882\n 195.623 196.388 212.753 221.361 222.05 223.976 224.553 224.841 258.309\n 260.155 264.838 270.837 298.139 298.321 310.671 326.34 326.805 327.084\n 333.681 334.333 334.95 337.978 339.219 339.686 347.099 347.633 349.095\n 349.095 549.983 552.372 553.674 553.85] not found in axis'

In [6]:
drop_extreme_cycles(central_cycle_df)

KeyError: '[43.5 59.165 69.341 69.438 70.125 71.052 71.812 71.951 72.392 72.392\n 79.438 91.74 92.5 92.5 112.886 152.174 156.801 156.985 164.407 164.407\n 164.883 171.814 173.542 174.182 174.956 175.96 178.078 185.223 187.094\n 187.608 188.873 190.302 191.454 191.794 193.198 193.589 194.882 194.882\n 195.623 196.388 212.753 221.361 222.05 223.976 224.553 224.841 258.309\n 260.155 264.838 270.837 298.139 298.321 310.671 326.34 326.805 327.084\n 333.681 334.333 334.95 337.978 339.219 339.686 347.099 347.633 349.095\n 349.095 549.983 552.372 553.674 553.85] not found in axis'

# UNUSED CODE

In [7]:
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))
import pandas as pd

import pandas as pd
import json
from analysis.bz_csd import bz_csd

# If cycle_df is not already loaded
cycle_df = pd.read_pickle('/home/isabella/Documents/isabella/jake/ephys_analysis/processed_data/arm_theta_cycle_df.pkl')

# Function to compute CSD from theta cycle dataframe
def process_lfp_traces_for_cycle(arm_cycle_dict):
    """
    Process the LFP traces for a given theta cycle.

    :param lfp_data: DataFrame containing LFP traces for all relevant channels.
    :return: Processed data (e.g., CSD) for the theta cycle.
    """
    for cycle_number, lfp_df in arm_cycle_dict.items():
        
        ## Get data from frame
        timestamps = lfp_df.columns.view('int64')/1e9 #Convert datetime stamps back into floats
        lfp_data = lfp_df.drop('Cycle Theta Phase', axis = 0).to_numpy().T
        phase_data = lfp_df.loc(axis = 0)['Cycle Theta Phase'].to_numpy()
        
        ## Compute CSD for individual theta cycle
        csd = bz_csd(lfp_data, plot_lfp = False, plot_csd = False)
        
        ## Compute gamma power for indiviual theta cycle
        min_f = 20 # Minimum frequency to compute for
        max_f = 80 # Maximum frequency to compute for
        frequencies = np.linspace(min_f, max_f, 100) 
        
        # Perform the wavelet transform
        wavelet_coeffs = complex_morlet_wavelet_transform(lfp_data, frequencies, lfp_sampling_rate)

        n_bins = 50  # Number of bins for theta phase
        bin_edges = np.linspace(0, 2 * np.pi, n_bins + 1)
        power_spectra = np.zeros((len(frequencies), n_bins))

    

        return csd

# Loop through t-maze trials in dataframe
for index, row in cycle_df.iterrows():
    
    # Extract session and trial information
    trial = index
    
    # Load serialised JSON-formatted dicts from dataframe
    serialised_central_cycle_dict = json.loads(row['Central Cycles'])
    serialised_return_cycle_dict = json.loads(row['Return Cycles'])
    
    # Deserialise to get back the dictionary structure of {cycle_number: LFP and theta phase data}
    central_cycle_dict = {cycle_number: pd.read_json(df_json, date_unit = 's') for cycle_number, df_json in serialised_central_cycle_dict.items()}
    return_cycle_dict = {cycle_number: pd.read_json(df_json, date_unit = 's') for cycle_number, df_json in serialised_central_cycle_dict.items()}
    
    print(central_cycle_dict.values([0])
    # process_lfp_traces_for_cycle(central_cycle_dict)
    
    raise TypeError
    
    


    # Function: loop through each traversal for each arm

        # Get LFP traces for all relevant channels in that traversal
        
        # Compute CSD & gamma power for the traversal
          
        # Discard first and last theta cycle from each traversal to avoid edge artefacts
          
        # Bin for theta phase and average across traversal
          
        # Write option to plot
        
        
    # Average theta phase vs CSD/gamma power for all traversals each arm in the trial
    
    # Plot CSD/ gamma power for each arm for that trial, including how many traversals & cycles are averaged for each

SyntaxError: invalid syntax (3623857586.py, line 64)

In [ ]:
obj.lfp_data[1]['data'].shape

## Get theta phase data

In [ ]:
from get_theta_frequencies import get_theta_frequencies
from get_theta_phase import get_theta_phase
import numpy as np

lfp_sampling_rate = 1000
channels_to_load = [15, 39, 38]

# Loop through all sessions
for index, obj in df_all_sessions['ephys_object'].iteritems():
    
    # Load LFP for channel 15 (0um), 39 (-300um) and 38 (-600um) for all trials
    # Roughly corresponds to s.p., s.r. and s.l.m ?
    obj.load_lfp(obj.trial_iterators, 
                 sampling_rate = lfp_sampling_rate, 
                 channels = channels_to_load, 
                 reload_flag = False)
                 
    # Loop through trials
    for i in obj.trial_iterators:
        
        # Find peak theta frequencies for each channel and make dict of {channel: theta_freq}
        theta_freqs = get_theta_frequencies(obj.lfp_data[i]['data'], lfp_sampling_rate)
        theta_freqs = dict(zip(channels_to_load, theta_freqs))
        
        # Initialise theta phase array
        theta_phase = [None] * len(channels_to_load)
        cycle_numbers = [None] * len(channels_to_load)
        
        # Loop through each included channel
        for channel_iterator, channel in enumerate(channels_to_load):
            
            # Get peak theta frequency
            theta_freq = theta_freqs[channel]
            
            # Get LFP data for channel
            lfp_data_for_channel = obj.lfp_data[i]['data'][:, channel_iterator]
            
            # Calculate theta phase
            theta_phase[channel_iterator], cycle_numbers[channel_iterator] = get_theta_phase(lfp_data_for_channel, lfp_sampling_rate, theta_freq)

            
        # Add theta phase data to object
        obj.lfp_data[i]['theta_phase'] = np.array(theta_phase).T
        # Add cycle numbers data to object
        obj.lfp_data[i]['cycle_numbers'] = np.array(cycle_numbers).T
        
    print(f'Theta phase loaded for channels {channels_to_load} from session {index}')
    print(cycle_numbers)
    print(theta_phase)

## Plot CSD across theta phase for each open field trial, save result to dataframe

## Plot theta phase against frequency power

In [ ]:
import matplotlib.pyplot as plt
from scipy.signal import morlet

def complex_morlet_wavelet_transform(signal, frequencies, fs):
    """
    Apply the complex Morlet wavelet transform to a signal for a range of frequencies.

    :param signal: The input signal (time series)
    :param frequencies: Array of frequencies for which to compute the transform
    :param fs: Sampling frequency of the input signal
    :return: An array of wavelet coefficients, time x frequencies
    """
    wavelet_coeffs = np.zeros((len(signal), len(frequencies)), dtype=complex)
    
    # Perform wavelet transform for each frequency
    for i, freq in enumerate(frequencies):
        # Calculate the wavelet for given frequency and sampling frequency
        wavelet = morlet(M=len(signal), w=5, s=freq / fs)
        # Convolve signal with the wavelet
        wavelet_coeffs[:, i] = np.convolve(signal, wavelet, mode='same')
    
    return wavelet_coeffs

def plot_wavelet_power_spectrum(wavelet_coeffs, times, frequencies):
    """
    Plot a time-resolved power spectrum from the wavelet coefficients.

    :param wavelet_coeffs: Wavelet coefficients as returned by complex_morlet_wavelet_transform
    :param times: Array of time points corresponding to the signal
    :param frequencies: Array of frequencies used in the wavelet transform
    """
    power_spectrum = np.abs(wavelet_coeffs) ** 2
    
    plt.figure(figsize=(10, 6))
    plt.pcolormesh(times, frequencies, power_spectrum.T, cmap='jet')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.colorbar(label='Power')
    plt.title('Time-Resolved Power Spectrum')
    plt.tight_layout()
    
    
# Define the frequency range for the analysis
frequencies = np.linspace(5, 50, 100)  # 100 frequencies from 20 to 200 Hz
t_min = 50000
t_max = 100000
lfp_data = obj.lfp_data[0]['data'][t_min:t_max,1]
theta_phase = obj.lfp_data[0]['theta_phase'][t_min:t_max,1]

# Perform the wavelet transform
wavelet_coeffs = complex_morlet_wavelet_transform(lfp_data, frequencies, lfp_sampling_rate)

n_bins = 50  # Number of bins for theta phase
bin_edges = np.linspace(0, 2 * np.pi, n_bins + 1)
power_spectra = np.zeros((len(frequencies), n_bins))

for i in range(n_bins):
    # Find indices for the current theta phase bin
    indices = np.where((theta_phase >= bin_edges[i]) & (theta_phase < bin_edges[i + 1]))[0]

    # Extract the wavelet coefficients for these indices
    selected_coeffs = wavelet_coeffs[indices, :]

    # Calculate the average power spectrum for this phase bin
    power_spectra[:, i] = np.mean(np.abs(selected_coeffs) ** 2, axis=0)
    
# Smooth data across phase bins
from scipy.ndimage import gaussian_filter1d
# Apply this to each frequency bin
sigma = 2  # Standard deviation for Gaussian kernel
smoothed_power_spectra = np.array([gaussian_filter1d(power_spectra[i, :], sigma) for i in range(power_spectra.shape[0])])


# Plotting
theta_bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
plt.figure(figsize=(10, 6))
plt.pcolormesh(theta_bin_centers, frequencies, smoothed_power_spectra, cmap='jet')
plt.xlabel('Theta Phase [radians]')
plt.ylabel('Frequency [Hz]')
plt.colorbar(label='Power')
plt.title('Theta Phase vs Frequency Power Spectrum')

# Adjust the x-axis to show multiples of pi
x_ticks = np.linspace(0, 2 * np.pi, 5)  # 0, pi/2, pi, 3pi/2, 2pi
x_labels = ['0', r'$\frac{\pi}{2}$', r'$\pi$', r'$\frac{3\pi}{2}$', r'$2\pi$']
plt.xticks(x_ticks, x_labels)

# Overlay a theta cycle
theta_cycle = np.cos(np.linspace(0, 2 * np.pi, 100))
theta_phase_values = np.linspace(0, 2 * np.pi, 100)
offset = np.median(frequencies)  # Adjust as per your plot
theta_cycle_normalized = theta_cycle * np.ptp(frequencies)/4 + offset
plt.plot(theta_phase_values, theta_cycle_normalized, color='white', linestyle='--', linewidth=2)

plt.show()

# # Plot the power spectrum
# plot_wavelet_power_spectrum(wavelet_coeffs, obj.lfp_data[1]['timestamps'][t_min:t_max], frequencies)

# plt.plot(obj.lfp_data[1]['timestamps'][t_min:t_max], (filtered_lfp[t_min:t_max]/300)+50, c = 'w')
# plt.show()

## Plot mean frequency power spectrogram across theta phase for each session for contact at 0um, -300um, -600um from 20-200Hz

In [ ]:
from scipy.signal import hilbert, firwin, filtfilt, argrelmax
import matplotlib.pyplot as plt

# Find peak theta frequencies for each channel and make dict of {channel: theta_freq}
theta_freqs = get_theta_frequencies(obj.lfp_data[0]['data'], lfp_sampling_rate)
theta_freqs = dict(zip(channels_to_load, theta_freqs))

peak_freq = theta_freqs[15]
filt_half_bandwidth = 2
lfp = obj.lfp_data[0]['data'][:,0]

low_freq = peak_freq - filt_half_bandwidth
high_freq = peak_freq + filt_half_bandwidth
filter_taps = firwin(round(lfp_sampling_rate) + 1, [low_freq, high_freq], pass_zero=False, window='blackman', fs=lfp_sampling_rate)
pad_length = min(3 * (len(filter_taps) - 1), len(lfp) - 1)
filtered_lfp = filtfilt(filter_taps, 1, lfp, padlen=pad_length)

# find theta peaks (this is 0)
maxima = argrelmax(filtered_lfp[:2000])[0]

cycles = np.lib.stride_tricks.as_strided(maxima, shape=(maxima.size - 1, 2), strides=(maxima.strides[0], maxima.strides[0]))
cycles

# for cycle in cycles:
#     plt.plot(obj.lfp_data[0]['data'][cycle[0]:cycle[1],0])

plt.plot(obj.lfp_data[0]['data'][:2000,0])
    
# plt.scatter(argrelmax(filtered_lfp[:2000]), [0]*cycles, c = 'r')

plt.plot(filtered_lfp[:2000])


In [ ]:
plt.plot(theta_phase[0][:1000])
plt.plot((filtered_lfp[:1000]/2000)+np.pi)

## Mean across all trials for open field